In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacexpert"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Load the logo:
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Blue:
colors = {
'background': '#87D653',
'text': '#0000FF'}

# Layout:
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Logo:
    html.Center(children=[
        html.A(html.Img(height=150,src='data:image/png;base64,{}'.format(encoded_image.decode())),
               href="https://www.snhu.edu")
    ]),
    
    # Unique identifier:
    html.H1('Pserkyt\'s Dash', style = {
        "textAlign":"center",
        "color":colors["text"]
    }),
    html.Hr(),
    
    # Filtering options:
    dcc.Dropdown(
        id='filtering-id',
        options=[
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'WILD'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'TRACK'},
        ],
        placeholder="Select rescue filter...",
    ),
    html.Hr(),
    
    # Data table:
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # User-friendly interactive data table features:
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single", # TODO: use "multi" when I figure out how to display multiple on map
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side:
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(id="graph-id"),
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Filtering callback:
@app.callback(Output('datatable-id', 'data'),
              [Input('filtering-id', 'value')])
def update_filter(value):
    if(value == None):
        df = pd.DataFrame.from_records(shelter.read({}))
    elif (value == 'WATER'):
        df = pd.DataFrame(list(shelter.read({
                "$and": [
                {
                        "$or": [
                                {"breed":"Labrador Retriever Mix"},
                                {"breed":"Chesapeake Bay Retriever"},
                                {"breed":"Newfoundland"}
                        ]
                },
                {
                        "animal_type":"Dog",
                        "age_upon_outcome_in_weeks": {
                                "$gte":26, "$lte":156
                                },
                        "sex_upon_outcome":"Intact Female",
                }
                ]})))
    elif (value == 'WILD'):
        df = pd.DataFrame(list(shelter.read({
                "$and": [
                {
                        "$or": [
                                {"breed":"German Shepherd"},
                                {"breed":"Alaskan Malamute"},
                                {"breed":"Old English Sheepdog"},
                                {"breed":"Siberian Husky"},
                                {"breed":"Rottweiler"}
                        ]
                },
                {
                        "animal_type":"Dog",
                        "age_upon_outcome_in_weeks": {
                                "$gte":26, "$lte":156
                                },
                        "sex_upon_outcome":"Intact Male",
                }
                ]})))
    elif (value == 'TRACK'):
        df = pd.DataFrame(list(shelter.read({
                "$and": [
                {
                        "$or": [
                                {"breed":"Doberman Pinscher"},
                                {"breed":"German Shepherd"},
                                {"breed":"Golden Retriever"},
                                {"breed":"Bloodhound"},
                                {"breed":"Rottweiler"}
                        ]
                },
                {
                        "animal_type":"Dog",
                        "age_upon_outcome_in_weeks": {
                                "$gte":20, "$lte":300
                                },
                        "sex_upon_outcome":"Intact Male",
                }
                ]})))
    return df.to_dict('records')

# This callback will highlight a row on the data table when the user selects it:
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Pie chart:
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    figure = px.pie(
                dff["breed"],
                names=dff["breed"]
            )
    return figure

# Geolocation:
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    loc_lat = dff["location_lat"].values[0]
    loc_long = dff["location_long"].values[0]
    animalName = dff["name"].values[0]
    animalBreed = dff["breed"].values[0]
    if selected_rows:
        loc_lat = dff["location_lat"].values[selected_rows[0]]
        loc_long = dff["location_long"].values[selected_rows[0]]
        animalName = dff["name"].values[selected_rows[0]]
        animalBreed = dff["breed"].values[selected_rows[0]]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[loc_lat,loc_long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[loc_lat,loc_long], children=[
                dl.Tooltip(animalBreed),
                dl.Popup([
                    html.H1("Animal Name:"),
                    html.P(animalName)
                ])
            ])
        ])
    ]

app

Database(MongoClient(host=['localhost:35793'], document_class=dict, tz_aware=False, connect=True), 'AAC')
